In [39]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

In [40]:
df = pd.read_csv("../../db/data/merge/tertiary/idxs_.csv",index_col=0)
df_logs = pd.read_csv("../../db/data/merge/tertiary/logs_.csv",index_col=0)

## *Exponential Moving Average*

In [25]:
def ema(df, window=14):
    """
    Exponential movint average
    """
    data = pd.DataFrame([])
    
    for currency in df.columns:
        data[f'{currency}_ema'] = df[currency].ewm(alpha=1/window, adjust=False).mean()
        
    data.index = df.index
    
    return data

## *Highpass Filter*

In [104]:
def highpass_filter(df, order=5, cutoff=0.2):
    """Highpass filter """
    
    data = pd.DataFrame([])
    
    b, a = sig.butter(N=order, Wn=cutoff, btype='highpass', analog=False)
    
    for currency in df.columns:
        data[f'{currency}_high'] = sig.lfilter(b, a, df[currency])
    
    data.index = df.index
    
    return data

## *Lowpass Filter*

In [106]:
def lowpass_filter(df, order=8, cutoff=0.2):
    """Lowpass filter """
    data = pd.DataFrame([])
    
    # filter
    b, a = sig.butter(N=order, Wn=cutoff, btype='lowpass', analog=False)
    
    for currency in df.columns:
        data[f'{currency}_low'] = sig.lfilter(b, a, df[currency])
    
    data.index = df.index
    
    return data


## *Relative Strenght Index*

In [84]:
def rsi(df, periods = 240):
    """Returns a series with the rsi """
    data = pd.DataFrame([])
    for currency in df.columns:
        delta = df[currency].diff()
        up = delta.clip(lower=0)
        down = -1*delta.clip(upper=0)
        
        ema_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ema_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
        rs = ema_up / ema_down
        rsi = 100 - (100/(1 + rs))
        
        data[f'{currency}_rsi'] = rsi
        
    return data